In [1]:
# file: marketing_attribution_regression.py

"""
Marketing Spend Attribution via Regression
- Supports Excel input OR synthetic data
- Linear Regression + LASSO
- Outputs ROI-style interpretation + budget guidance
"""

from __future__ import annotations

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import statsmodels.api as sm


# -----------------------------
# Data Loading
# -----------------------------
def load_data(
    excel_path: str | None = None,
    sheet_name: str = "data",
) -> pd.DataFrame:
    """
    Expected columns:
    date | conversions | search_spend | social_spend | display_spend | email_spend
    """
    if excel_path:
        df = pd.read_excel(excel_path, sheet_name=sheet_name)
    else:
        df = generate_sample_data()

    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").set_index("date")
    return df


def generate_sample_data(n_periods: int = 104) -> pd.DataFrame:
    rng = np.random.default_rng(42)
    dates = pd.date_range("2023-01-01", periods=n_periods, freq="W")

    search = rng.normal(5000, 800, n_periods)
    social = rng.normal(3500, 600, n_periods)
    display = rng.normal(2000, 500, n_periods)
    email = rng.normal(1200, 300, n_periods)

    conversions = (
        0.08 * search +
        0.05 * social +
        0.02 * display +
        0.12 * email +
        rng.normal(0, 300, n_periods)
    )

    return pd.DataFrame(
        {
            "date": dates,
            "conversions": conversions.clip(min=0),
            "search_spend": search.clip(min=0),
            "social_spend": social.clip(min=0),
            "display_spend": display.clip(min=0),
            "email_spend": email.clip(min=0),
        }
    )


# -----------------------------
# Cleaning
# -----------------------------
def clean_timeseries(df: pd.DataFrame) -> pd.DataFrame:
    # Fill missing dates
    df = df.asfreq("W")

    # Fill gaps conservatively
    df = df.interpolate(method="linear").ffill().bfill()

    # Winsorize outliers (robust, avoids arbitrary removal)
    for col in df.columns:
        lower, upper = df[col].quantile([0.01, 0.99])
        df[col] = df[col].clip(lower, upper)

    return df


# -----------------------------
# Modeling
# -----------------------------
def fit_ols(X: pd.DataFrame, y: pd.Series):
    X_const = sm.add_constant(X)
    model = sm.OLS(y, X_const).fit()
    return model


def fit_lasso(X: pd.DataFrame, y: pd.Series):
    pipeline = Pipeline(
        steps=[
            ("scaler", StandardScaler()),
            ("lasso", LassoCV(cv=5, random_state=42)),
        ]
    )
    pipeline.fit(X, y)
    return pipeline


# -----------------------------
# Attribution + ROI
# -----------------------------
def compute_roi(coeffs: pd.Series, mean_spend: pd.Series) -> pd.DataFrame:
    roi = coeffs / mean_spend
    return pd.DataFrame(
        {
            "coefficient": coeffs,
            "mean_spend": mean_spend,
            "roi_proxy": roi,
        }
    ).sort_values("roi_proxy", ascending=False)


# -----------------------------
# Summary
# -----------------------------
def generate_summary(roi_df: pd.DataFrame) -> str:
    top = roi_df.index[0]
    bottom = roi_df.index[-1]

    return f"""
MARKETING ATTRIBUTION SUMMARY
-----------------------------
Model: Linear Regression + LASSO validation

Top Performing Channel:
- {top}
- Highest marginal conversions per $ spent

Lowest Performing Channel:
- {bottom}

Recommendation:
- Reallocate 10–20% budget from '{bottom}' to '{top}'
- Maintain spend on mid-tier channels with stable ROI
- Re-evaluate quarterly as saturation effects may appear

Interpretation Note:
- Coefficients represent marginal contribution assuming linearity
- LASSO confirms robustness under multicollinearity
""".strip()


# -----------------------------
# Main
# -----------------------------
def main(excel_path: str | None = None):
    df = load_data(excel_path)
    df = clean_timeseries(df)

    y = df["conversions"]
    X = df.drop(columns="conversions")

    # OLS
    ols_model = fit_ols(X, y)
    ols_coeffs = ols_model.params.drop("const")

    # LASSO
    lasso_model = fit_lasso(X, y)
    lasso_coeffs = pd.Series(
        lasso_model.named_steps["lasso"].coef_,
        index=X.columns,
    )

    roi = compute_roi(ols_coeffs, X.mean())

    print("\n=== OLS Regression Summary ===\n")
    print(ols_model.summary())

    print("\n=== ROI Proxy (OLS) ===\n")
    print(roi.round(4))

    print("\n=== LASSO Coefficients ===\n")
    print(lasso_coeffs.round(4))

    print("\n=== EXECUTIVE SUMMARY ===\n")
    print(generate_summary(roi))


if __name__ == "__main__":
    main()  # pass Excel path if available



=== OLS Regression Summary ===

                            OLS Regression Results                            
Dep. Variable:            conversions   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     2.868
Date:                Thu, 15 Jan 2026   Prob (F-statistic):             0.0270
Time:                        13:05:30   Log-Likelihood:                -739.72
No. Observations:                 104   AIC:                             1489.
Df Residuals:                      99   BIC:                             1503.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const        